# Description: 
* **DOMAIN:** Smartphone, Electronics  

* **CONTEXT:** India is the second largest market globally for smartphones after China. About 134 million smartphones were sold across India in the year 2017 and is estimated to increase to about 442 million in 2022. India ranked second in the average time spent on mobile web by smartphone users across Asia Pacific. The combination of very high sales volumes and the average smartphone consumer behaviour has made India a very attractive market for foreign vendors. As per Consumer behaviour, 97% of consumers turn to a search engine when they are buying a product vs. 15% who turn to social media. If a seller succeeds to publish smartphones based on user’s behaviour/choice at the right place, there are 90% chances that user will enquire for the same. This Case Study is targeted to build a recommendation system based on individual consumer’s behaviour or choice.

* **DATA DESCRIPTION:**  
• author : name of the person who gave the rating  
• country : country the person who gave the rating belongs to  
• data : date of the rating  
• domain: website from which the rating was taken from  
• extract: rating content  
• language: language in which the rating was given  
• product: name of the product/mobile phone for which the rating was given  
• score: average rating for the phone  
• score_max: highest rating given for the phone  
• source: source from where the rating was taken  

* **PROJECT OBJECTIVE:** We will build a recommendation system using popularity based and collaborative filtering methods to recommend mobile phones to a user which are most popular and personalised respectively.

## Table of Contents <a id="toc"></a>
* [# Import Libraries](#import_libraries)  
1. [Data Prepration and cleaning](#data_prepration)  
    1.1. [Load the dataset](#load_data)  
    1.2. [Merge dataset](#merge_data)  
    1.3. [Basic Analysis](#basic_analysis)  
    1.4. [Modification 1: revs1: Data cleaning, Imputation and rounding-off](#clean_impute_round-off)  
    1.5. [Data split](#data_split)  
2. [Analysis](#analysis)
3. [Recommend top 5 mobile phones using popularity based model](#popularity_model)
4. [Collaborative filtering based models](#collaborative_filtering)  
    4.1 [SVD](#svd)  
    4.2 [kNNWithMeans_Item based](#knnwithmeans_item_based)  
    4.3 [kNNWithMeans_User based](#knnwithmeans_user_based)  
5. [Show RMSE value and comparison](#rmse)  
6. [Average ratings for test users](#average_rating)  
7. [Summary (findings and Inferences)](#summary_inferences)
8. [Recommend top 5 products for test users](#recommend_top_5)  
9. [Results with cross_validation techniques](#cross_validation)  
10. [In what business scenario you should use popularity based Recommendation Systems?](#q10)  
11. [In what business scenario you should use CF based Recommendation Systems ?](#q11)
12. [What other possible methods can you think of which can further improve the recommendation for different users ?](#q12)

## #. Import libraries <a id="import_libraries"></a>
[Go to top](#toc)

[Go to top](#toc)

In [ ]:
# Main libraries
import os
import pandas as pd
import numpy as np

In [ ]:
# Visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# Libraries for recommendation systems
from collections import defaultdict
from surprise import SVD
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

In [ ]:
!pip install translate
!pip install google-trans-new

In [ ]:
from google_trans_new import google_translator
from translate import Translator

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option("display.max_rows", 50, "display.max_columns", 50)
pd.set_option('display.max_colwidth', None)
plt.style.use('bmh')
# create contants
RS=612

## 1. Data preparation and basic cleaning <a id="data_prepration"></a>
[Go to top](#toc)

[Go to top](#toc)

### 1.1.  Load the dataset <a id="load_data"></a>
[Go to top](#toc)

In [ ]:
%%time
#Loading Data files
review_1 = pd.read_csv('../input/recommendation-system/phone_user_review_file_1.csv', encoding='iso-8859-1')
review_2 = pd.read_csv('../input/recommendation-system/phone_user_review_file_2.csv', encoding='iso-8859-1')
review_3 = pd.read_csv('../input/recommendation-system/phone_user_review_file_3.csv', encoding='iso-8859-1')
review_4 = pd.read_csv('../input/recommendation-system/phone_user_review_file_4.csv', encoding='iso-8859-1')
review_5 = pd.read_csv('../input/recommendation-system/phone_user_review_file_5.csv', encoding='iso-8859-1')
review_6 = pd.read_csv('../input/recommendation-system/phone_user_review_file_6.csv', encoding='iso-8859-1')

In [ ]:
review_1.head()

In [ ]:
review_2.head()

In [ ]:
print(f'review_1: Rows: {review_1.shape[0]} and Columns: {review_1.shape[1]}\n')
print(f'review_2: Rows: {review_2.shape[0]} and Columns: {review_2.shape[1]}\n')
print(f'review_3: Rows: {review_3.shape[0]} and Columns: {review_3.shape[1]}\n')
print(f'review_4: Rows: {review_4.shape[0]} and Columns: {review_4.shape[1]}\n')
print(f'review_5: Rows: {review_5.shape[0]} and Columns: {review_5.shape[1]}\n')
print(f'review_6: Rows: {review_6.shape[0]} and Columns: {review_6.shape[1]}\n')
print(f'Total rows: {review_1.shape[0]+review_2.shape[0]+review_3.shape[0]+review_4.shape[0]+review_5.shape[0]+review_6.shape[0]}')

In [ ]:
print('Check whether the column names are same in all the dataframes: ')
all(np.unique(review_1.columns.tolist()) == np.unique(review_1.columns.tolist()+
                                                      review_2.columns.tolist()+
                                                      review_3.columns.tolist()+
                                                      review_4.columns.tolist()+
                                                      review_5.columns.tolist()+
                                                      review_6.columns.tolist()))

### 1.2. Merge dataset <a id="merge_data"></a>
[Go to top](#toc)

In [ ]:
#Merge the data into a single dataframe 
reviews = pd.concat([review_1,review_2,review_3,review_4,review_5,review_6], ignore_index=True)
del review_1, review_2, review_3, review_4, review_5, review_6
print(f'reviews: Rows: {reviews.shape[0]} and Columns: {reviews.shape[1]}\n')
print('Top 5 rows of the data: ')
display(reviews.head())
print('Bottom 5 rows of the data: ')
display(reviews.tail())

In [ ]:
reviews.info()

* Except **score** and **score_max** (which are of float type) all other features are of object type 
* feature **date** should be of datetype
* Also, **score, score_max, extract and author**: columns seems to have Null values

### 1.3. Basic analysis <a id="basic_analysis"></a>
[Go to top](#toc)

In [ ]:
df=reviews.isna().sum().round(2)
df1 = (df*100/reviews.shape[0]).round(2)
print('Missing count and percentages for each column are: \n',df.astype('str') +' ('+ df1.astype('str')+'%)')

#fig = px.bar(x=df.index, y=df1,text = df.astype('str') +'('+ df1.astype('str')+'%)',
#             title="Count (text) and Percentage(yaxis) of missing values in all the features (data: reviews)")
#fig.update_xaxes(title_text= 'Features')
#fig.update_yaxes(title_text= 'Percentage of Missing values')
#fig.show()
del df, df1

Thus these three features have approx 4.5% missing values
'score' and 'score_max' have exactly same number of missing values

In [ ]:
print('Number of unique values in each feature: \n',reviews.nunique())

i.e. score_max for all the phones is 10 throughout.

In [ ]:
# Top 10 Non-english users
reviews[reviews['lang']!='en']['author'].value_counts(ascending=False)[:10]

In [ ]:
# Top 10 Non-english products
reviews[reviews['lang']!='en']['product'].value_counts(ascending=False)[:10]

Thus, a multiple similar names, with different details exist in product list. For eg:  
* Huawei P8lite zwart / 16 GB and  
* Huawei P8 Lite Smartphone, Display 5" IPS, Processore Octa-Core 1.5 GHz, Memoria Interna da 16 GB, 2 GB RAM, Fotocamera 13 MP, monoSIM, Android 5.0, Bianco [Italia]  
are exactly same models

Another observation is that 'phone_url' column also contains the phone name and model information. Let's check what extra information is present in 'product column'

In [ ]:
# print the frequency count of phone_url column
reviews['phone_url'].value_counts(ascending=False).head(50)

In [ ]:
# frequency count of 'product' column for 'samsung-galaxy-s-iii' type phone
reviews[reviews["phone_url"]=='/cellphones/samsung-galaxy-s-iii/'][['product']].value_counts().head(15)

Exra information is generally:  
* phone memory: 8Gb/16GB/32GB etc
* phone colour: Marble white, Blue, Red etc
* carrier: AT&T, Verizon etc

Another observation is that these specifications are not present in all the product names, for eg: 
there is no-way available to differentiate between the 2 products below:  
'Samsung Galaxy S III Cellular Phone' and   
'Samsung Galaxy S III SPH-L710 - 16GB - Marble White (Sprint) Smartphone'

Thus differentiating information is not same in all the product details. 
Also, the goal is to recommend a phone not the carrier. and other specs like color etc are of low importance in recommendation. The only consistent differentiating information in all the product names is the 'phone manufacturer and model number', which can also be extracted from 'phone_url' column.
Let's check for other phone names as well

In [ ]:
# frequency count of 'product' column for 'apple-iphone-5s' type phone
reviews[reviews["phone_url"]=='/cellphones/apple-iphone-5s/'][['product']].value_counts().head(15)

In [ ]:
# frequency count of 'product' column for 'samsung-galaxy-s6' type phone
reviews[reviews["phone_url"]=='/cellphones/samsung-galaxy-s6/'][['product']].value_counts().head(15)

In [ ]:
# frequency count of 'product' column for 'samsung-galaxy-s5' type phone
reviews[reviews["phone_url"]=='/cellphones/samsung-galaxy-s5/'][['product']].value_counts().head(15)

As can be seen, same pattern is visible for the most comun types of phones. Thus it is better to use phone name and model number rather than other details mentioned in 'product' column

In [ ]:
reviews['phone'] = reviews['phone_url'].str.split("/").apply(lambda col: col[2]).replace('-', ' ', regex=True)
reviews['product'] = reviews['phone']
reviews['phone'].unique()

In [ ]:
reviews['product'].shape

In [ ]:
product = reviews['product'].value_counts()[:10]
print('Distribution of number of ratings per item (Clipped at 10): \n',product)
sns.barplot(y=product.index,x=product)
plt.tight_layout()
plt.show()

In [ ]:
users = reviews['author'].value_counts(dropna=False)[:10]
print('Distribution of number of ratings per user(Clipped at 10): \n',users)
users.index = users.index.map(str)
sns.barplot(y=users.index,x=users)
plt.tight_layout()
plt.show()

Following observations are made:  
1. Most active user is 'Amazon customer'
2. 'Anonymous' and 'unknown' users are those whose names are not known. Thus we can use this to impute blank values in 'author' column
3. Many names are similar but in different languages like 'Amazon customer' and 'Cliente Amazon'. Let's search for these first and cleanup the differences due to language

names like 'einer Kundin', 'einem Kunden','Anonymous' and 'unknown' can be interpreted in the same way i.e. an 'unknown customer'. Let's replace these names too

In [ ]:
unknowns = ['Anonymous','einer Kundin','einem Kunden', 'unknown','Anonymous ']
reviews['author'].replace(to_replace = unknowns, 
                          value = 'Anonymous', 
                          inplace=True)

In [ ]:
users = reviews['author'].value_counts(dropna=False)[:10]
print('Distribution of number of ratings per user(Clipped at 10): \n',users)
users.index = users.index.map(str)
sns.barplot(y=users.index,x=users)
plt.tight_layout()
plt.show()

In [ ]:
# Let's check the score column of the data
print('Uniqe values in the "score" feature: \n',reviews.score.unique())

Let's round it off to nearest integer

In [ ]:
relevant_features=['author','product','score']
# irrelvant_features=['phone_url','date','lang','country','source','domain','score_max','extract']

In [ ]:
print('Number of duplicate rows: ', reviews.duplicated().sum())

In [ ]:
orgnl_rows = reviews.shape[0]
orgnl_columns = reviews.shape[1]

### 1.4. Modification 1: revs1: Data cleaning, Imputation and rounding-off <a id="clean_impute_round-off"></a>
[Go to top](#toc)

In [ ]:
revs1 = reviews.copy()

# Delete data which is not useful anymore, to save memory
del reviews

# Step1: remove irrelevant features
revs1 = revs1.loc[:,relevant_features]
print(f'Step1: revs1 Shape after removing irrelevant features: Rows: {revs1.shape[0]} and Columns: {revs1.shape[1]}\n')

# Step2: Round-off score feature to nearest integer
revs1['score'] = revs1['score'].round(0).astype('Int64')
print('Step2: Round-off: Unique values in the "score" feature(after rounding-off): \n',list(revs1.score.unique()))

# Step3: Impute missing values in score feature with median
revs1['score'] = revs1['score'].fillna(revs1['score'].median())
print('\nStep3: Imputation of "score"  with median and "author" with "Anonymous"')

# Step4: remove samples with missing values in 'Product' and 'author' feature and also 'Anonymous' values
revs1.dropna(inplace=True)
revs1 = revs1[revs1["author"] != 'Anonymous']
print(f'\nStep4: revs1 Shape(after removing missing values): Rows: {revs1.shape[0]} and Columns: {revs1.shape[1]}\n')

# Step5: remove duplicates, if any
revs1 = revs1.drop_duplicates()
print(f'Step5: revs1 Shape(after removing duplicates): Rows: {revs1.shape[0]} and Columns: {revs1.shape[1]}\n')

print(f'Overall {round(100 - revs1.shape[0]*100/orgnl_rows,2)}% samples are dropped\n')
revs1.head()

### 1.5. Data split <a id="data_split"></a>
[Go to top](#toc)

In [ ]:
# separate 1 million data samples
revs_1m = revs1.sample(n=1000000, random_state=RS)
print(f'revs2 Shape: Rows: {revs_1m.shape[0]} and Columns: {revs_1m.shape[1]}\n')

## 2. Analysis <a id="analysis"></a>
[Go to top](#toc)

In [ ]:
# 1. Most rated features
print('Most rated features/products: \n\n',revs_1m['product'].value_counts().head())

In [ ]:
# find out which rating is given highest number of times
sns.countplot(data=revs_1m , x='score')
plt.show()

In [ ]:
revs1[revs1['score']==10]['author'].value_counts().head(10)

In [ ]:
# 2. Users with highest number of reviews
print('Users with highest number of reviews: \n\n',revs_1m['author'].value_counts().head())

In [ ]:
# 3. Select data with products having >50 ratings and users who have given > 50 ratings
author50 = revs1['author'].value_counts()
author50 = author50[author50>50].index.tolist() # list of authors with > 50 ratings
print('Number of authors who have given >50 rating: ', len(author50))

product50 = revs1['product'].value_counts()
product50 = product50[product50>50].index.tolist() # list of products with > 50 ratings
print('Number of products with >50 rating: ', len(product50))

revs_50 = revs1[(revs1['author'].isin(author50)) & (revs1['product'].isin(product50))]
print(f'\nrevs_50: Rows: {revs_50.shape[0]} and Columns: {revs_50.shape[1]}\n')
del author50, product50
revs_50.head()

In [ ]:
revs_50['author'].unique()[:100]

In [ ]:
revs_50['product'].unique()[:100]

In [ ]:
#target = 3000
top50_product = revs1['product'].value_counts()[0:50].rename('rating_count').to_frame()
top50_product['mean_ratings']=revs1[revs1['product'].isin(top50_product.index.tolist())].groupby(['product'])['score'].mean().astype('float64').round(1)
top50_product.sort_values(by='mean_ratings',inplace=True)
print('Number of products with >'+str(target)+' rating: ', len(top50_product))

In [ ]:
fig = px.scatter(top50_product, x=top50_product.index, y="mean_ratings", size="rating_count", size_max=60,
                  height=800,title="Visualisation of mean ratings vs rating count for highest rated 50 phones")
fig.show()

## 3. Recommend top 5 mobile phones using popularity based model <a id="popularity_model"></a>
[Go to top](#toc)

In [ ]:
def popularity_rec(data):
    ratings_mean_count = pd.DataFrame(data.groupby('product')['score'].mean())
    ratings_mean_count['rating_counts'] = data.groupby('product')['score'].count()
    ratings_mean_count = ratings_mean_count.sort_values(by=['score','rating_counts'], ascending=[False,False])
    print('Top 5 recommendations for the products are: \n')
    display(ratings_mean_count.head())
    return

In [ ]:
# Using the data from the most popular phones amongst the most frequent users
popularity_rec(revs_50)

In [ ]:
# if we consider the original data (excluding 'Anonymous' users)
popularity_rec(revs1)

In [ ]:
field_length = revs1.author.astype(str).map(len)
print (revs1.iloc[field_length.argmax(),0])

## 4. Collaborative filtering based models <a id="collaborative_filtering"></a>
[Go to top](#toc)

### 4.1. Collaborative filtering model using SVD <a id="svd"></a>
[Go to top](#toc)

In [ ]:
# Rearrange columns for SVD and prepare train and testsets
revs50_ = Dataset.load_from_df(revs_50[['author','product','score']], Reader(rating_scale=(1, 10)))
trainset, testset = train_test_split(revs50_, test_size=.25,random_state=RS)

print('top 3 values from trainset: \n')
for key,value in {k: v for k, v in trainset.ur.items() if k <= 2}.items(): print(key,'-> ',value,'\n')
print('\ntop 3 values from testset: ', *testset[0:3], sep='\n\n')

In [ ]:
# Objective: To get top_n recommendation for each user
def get_top_n(predictions, n=5):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the n highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
%%time
# fit and predict using svd
def svd_func(train, test):
    svd = SVD(random_state=RS)
    svd.fit(train)
    svd_pred = svd.test(test)
    return svd_pred, svd

svd_pred, svd = svd_func(trainset,testset)
print('First few prediction values: \n',svd_pred[0:2])
print('\nRMSE value(test-set): ',round(accuracy.rmse(svd_pred),2),'\n') # compute RMSE
svd_rmse = round(accuracy.rmse(svd_pred),2)

### 4.2. Collaborative filtering model using kNNWithMeans_Item based <a id="knnwithmeans_item_based"></a>
[Go to top](#toc)

In [ ]:
%%time
# fit and predict using knn
def knn_item(train, test):
    knn_i = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
    knn_i.fit(train)
    knn_i_pred = knn_i.test(test)
    return knn_i_pred, knn_i

knn_i_pred, knn_i = knn_item(trainset, testset)
print('First few prediction values: \n',knn_i_pred[0:2])
print('\nRMSE value(Item-based Model, test-set): ',round(accuracy.rmse(knn_i_pred),2),'\n') # compute RMSE
knn_i_rmse = round(accuracy.rmse(knn_i_pred),2)

## 4.3. Collaborative filtering model using kNNWithMeans_User based <a id="knnwithmeans_user_based"></a>
[Go to top](#toc)

In [ ]:
%%time
# fit and predict using knn
def knn_user(train, test):
    knn_u = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
    knn_u.fit(train)
    knn_u_pred = knn_u.test(test)
    return knn_u_pred, knn_u

knn_u_pred, knn_u = knn_user(trainset, testset)
print('First few prediction values: \n',knn_u_pred[0:2])
print('\nRMSE value(User-based Model, test-set): ',round(accuracy.rmse(knn_u_pred),2),'\n') # compute RMSE
knn_u_rmse = round(accuracy.rmse(knn_u_pred),2)

## 5. Show RMSE value and comparison <a id="rmse"></a>
[Go to top](#toc)

In [ ]:
# Comparison of RMSE scores from different collaorative algorithms
sns.barplot(x=['svd_rmse','knn_i_rmse', 'knn_u_rmse'],y=[svd_rmse,knn_i_rmse, knn_u_rmse])

Best RMSE score is given by knn (item based), so let's use it for further analyssi

## 6. Average ratings for test users <a id="average_rating"></a>
[Go to top](#toc)

In [ ]:
svd_pred_df=pd.DataFrame(svd_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])
print('average prediction for test users: ',svd_pred_df['est'].mean())
print('average rating  by test users: ',svd_pred_df['rui'].mean())
print('average prediction error for test users: ',(svd_pred_df['rui']-svd_pred_df['est']).abs().mean())

In [ ]:
knn_i_pred_df=pd.DataFrame(knn_i_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])
print('average prediction for test users: ',knn_i_pred_df['est'].mean())
print('average rating  by test users: ',knn_i_pred_df['rui'].mean())
print('average prediction error for test users: ',(knn_i_pred_df['rui']-knn_i_pred_df['est']).abs().mean())

In [ ]:
knn_u_pred_df=pd.DataFrame(knn_u_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])
print('average prediction for test users: ',knn_u_pred_df['est'].mean())
print('average rating  by test users: ',knn_u_pred_df['rui'].mean())
print('average prediction error for test users: ',(knn_u_pred_df['rui']-knn_u_pred_df['est']).abs().mean())

## 7. Summary (findings and Inferences) <a id="summary_inferences"></a>
[Go to top](#toc)

1. Most popular phone (rated 10 by highest number of people):  
        * Overall: verykool t742
        * Amongst top users: samsung e1120       
2. Overall data is highly skewed towards 'Amazon customers' from different countries. This may also be because 'Amazon' is the biggest trader for phones in the world. Although correct 'user' names from 'Amazon' should have used.
3. Most of the authors have given the rating of '10' or '8'
4. Both knn_i(item-based) and knn_u(user-based) have roughly similar RMSE

## 8. Recommend top 5 products for test users <a id="recommend_top_5"></a>
[Go to top](#toc)

In [ ]:
%%time
#recommend top 5 products for test users
top_5 = get_top_n(knn_i_pred,5)
print('Top 5 recommendations for all test users are: \n')
for key,value in top_5.items(): print(key,'-> ',value,'\n') # to print all the recommendations for all the users
#print('Top 5 recommendations for 3 users are: \n')
#for key,value in {k: v for k, v in top_5.items() if k in ['Amazon Customer','Cliente Amazon',"Client d'Amazon"]}.items(): print(key,'-> ',value,'\n')

## 9. Results with cross_validation techniques <a id="cross_validation"></a>
[Go to top](#toc)

In [ ]:
%%time
svd_cv = cross_validate(svd,revs50_, measures=['RMSE'], cv=5, verbose=False)
print('\n Mean svd cv score:', round(svd_cv['test_rmse'].mean(),2),'\n')
svd_cv

In [ ]:
%%time
knn_i_cv = cross_validate(knn_i,revs50_, measures=['RMSE'], cv=5, verbose=False)
print('\n Mean knn_i_cv score:', round(knn_i_cv['test_rmse'].mean(),2),'\n')
knn_i_cv

In [ ]:
%%time
knn_u_cv = cross_validate(knn_u,revs50_, measures=['RMSE'], cv=5, verbose=False)
print('\n Mean knn_u_cv score:', round(knn_u_cv['test_rmse'].mean(),2),'\n')
knn_u_cv

In [ ]:
# Comparison of RMSE scores(mean cv) from different collaorative algorithms
sns.barplot(y=['svd_cv_rmse','knn_i_cv_rmse', 'knn_u_cv_rmse'],
            x=[svd_cv['test_rmse'].mean(),knn_i_cv['test_rmse'].mean(), knn_u_cv['test_rmse'].mean()])

Thus, for cv scores too, knn_i is giving a better performance

**10: In what business scenario you should use popularity based Recommendation Systems ?** <a id="q10"></a>
[Go to top](#toc)

> Popularity based recommendation systems can be useful in multiple scenarios like:  
> 1. When there is no data about the user and items.
> 2. When it is required to show most popular items in different categories along with personalized results like: 
>     * Most popular punjabi songs or most popular english songs on a music website/app
>     * Most popular trend in cwestern wear or traditional wear
>     * Most popular holiday packages for honeymoon trips, or bike trips or himalayan trips etc

**11: In what business scenario you should use CF based Recommendation Systems ?**  <a id="q11"></a>
[Go to top](#toc)
> Collaborative filtering is useful in scenarios like: 
> 1. Giving personalised recommendation to the user, when user history or item data is available. Some examples can be:
>     * Personalized movie recommendation of movie sites like Netflix, Amazon Prime, Youtube etc

**12: What other possible methods can you think of which can further improve the recommendation for different users ?**  <a id="q12"></a>
[Go to top](#toc)
> Other from Popularity and Collaborative Filtering, hybrid recommendation methods like Content+Collaborative method, Demographic, Utility based, and Knowledge based recommendation system can also be used.